In [82]:
import numpy as np
import pandas as pd

In [83]:
full_df = pd.read_csv('data/test_table.csv')
user_df = pd.read_csv('data/user_table.csv')

In [84]:
total_df = pd.merge(full_df, user_df, on='user_id', how='inner')

In [5]:
print full_df.columns
print user_df.columns

Index([u'user_id', u'date', u'source', u'device', u'browser_language',
       u'ads_channel', u'browser', u'conversion', u'test'],
      dtype='object')
Index([u'user_id', u'sex', u'age', u'country'], dtype='object')


In [70]:
total_df.head()

,user_id,date,source,device,browser_language,ads_channel,browser,conversion,test,sex,age,country
0,315281,2015-12-03,Direct,Web,ES,NaN,IE,1,0,M,32.0,Spain
1,497851,2015-12-04,Ads,Web,ES,Google,IE,0,1,M,21.0,Mexico
2,848402,2015-12-04,Ads,Web,ES,Facebook,Chrome,0,0,M,34.0,Spain
3,290051,2015-12-03,Ads,Mobile,Other,Facebook,Android_App,0,1,F,22.0,Mexico
4,548435,2015-11-30,Ads,Web,ES,Google,FireFox,0,1,M,19.0,Mexico


In [39]:
full_df.head()

,user_id,date,source,device,browser_language,ads_channel,browser,conversion,test
0,315281,2015-12-03,Direct,Web,ES,NaN,IE,1,0
1,497851,2015-12-04,Ads,Web,ES,Google,IE,0,1
2,848402,2015-12-04,Ads,Web,ES,Facebook,Chrome,0,0
3,290051,2015-12-03,Ads,Mobile,Other,Facebook,Android_App,0,1
4,548435,2015-11-30,Ads,Web,ES,Google,FireFox,0,1


In [40]:
user_df.head()

,user_id,sex,age,country
0,765821,M,20,Mexico
1,343561,F,27,Nicaragua
2,118744,M,23,Colombia
3,987753,F,27,Venezuela
4,554597,F,20,Spain


These next few cells are testing integrity of the data

In [77]:
print len(full_df), len(user_df), len(total_df)
print len(full_df), len(full_df['user_id'].unique())

453321 452867 452867
453321 453321


In [19]:
sum(full_df['conversion'].isnull())

0

In [25]:
full_df['browser_language'].unique()

array(['ES', 'Other', 'EN'], dtype=object)

Splitting the data into control and test dataframes, and those into successes and failures

In [85]:
control_df = full_df[full_df['test']==0]
test_df = full_df[full_df['test']==1]
print len(control_df), len(test_df)

237338 215983


In [105]:
control_successes = control_df[control_df['conversion']==1]
control_failures = control_df[control_df['conversion']==0]
test_successes = test_df[test_df['conversion']==1]
test_failures = test_df[test_df['conversion']==0]

print float(len(control_successes))/len(control_failures)*100
print float(len(test_successes))/len(test_failures)*100
print 'control successes : {}'.format(len(control_successes)), 'control failures : {}'.format(len(control_failures))
print 'test successes : {}'.format(len(test_successes)), 'test failures : {}'.format(len(test_failures))


5.84011915698
4.53960233103
control successes : 13096 control failures : 224242
test successes : 9379 test failures : 206604


Some more EDA on the control and test groups

In [46]:
print test_df['date'].min(), test_df['date'].max()
print control_df['date'].min(), control_df['date'].max()

2015-11-30 2015-12-04
2015-11-30 2015-12-04


In [47]:
# no significant anomlies in these columns: source, ads channel, browser, 
# the language could make a bit of a difference, less spanish in control group, not a huge difference
print control_df.groupby('device').sum()['conversion'], control_df.groupby('device').count()['conversion']
print test_df.groupby('device').sum()['conversion'], test_df.groupby('device').count()['conversion']

device
Mobile    5870
Web       7226
Name: conversion, dtype: int64 device
Mobile    105400
Web       131938
Name: conversion, dtype: int64
device
Mobile    4240
Web       5139
Name: conversion, dtype: int64 device
Mobile     96356
Web       119627
Name: conversion, dtype: int64


In [87]:
control_df2 = total_df[total_df['test']==0]
test_df2 = total_df[total_df['test']==1]
control_successes2 = control_df[control_df['conversion']==1]
control_failures2 = control_df[control_df['conversion']==0]
test_successes2 = test_df[test_df['conversion']==1]
test_failures2 = test_df[test_df['conversion']==0]

In [88]:
print control_df2.groupby('country').sum()['conversion'], control_df2.groupby('country').count()['conversion']
print test_df2.groupby('country').sum()['conversion'], test_df2.groupby('country').count()['conversion']

country
Argentina       141
Bolivia         274
Chile           474
Colombia       1411
Costa Rica      139
Ecuador         395
El Salvador     220
Guatemala       386
Honduras        222
Mexico         3178
Nicaragua       180
Panama           92
Paraguay        177
Peru            842
Spain          4128
Uruguay           5
Venezuela       813
Name: conversion, dtype: int64 country
Argentina       9356
Bolivia         5550
Chile           9853
Colombia       27088
Costa Rica      2660
Ecuador         8036
El Salvador     4108
Guatemala       7622
Honduras        4361
Mexico         64209
Nicaragua       3419
Panama          1966
Paraguay        3650
Peru           16869
Spain          51782
Uruguay          415
Venezuela      16149
Name: conversion, dtype: int64
country
Argentina       513
Bolivia         267
Chile           507
Colombia       1364
Costa Rica      145
Ecuador         385
El Salvador     195
Guatemala       365
Honduras        200
Mexico         3290
Nicaragua       1

In [106]:
control_successes = control_df2.groupby('country').sum()['conversion']
control_total = control_df2.groupby('country').count()['conversion']
test_successes = test_df2.groupby('country').sum()['conversion']
test_total = test_df2.groupby('country').count()['conversion']
for country in test_df2['country'].unique():
    print 'control',country, float(control_successes.loc[country]) / control_total.loc[country]*100
    print 'test',country, float(test_successes.loc[country]) / test_total.loc[country]*100

control Mexico 4.94946191344
test Mexico 5.11863088292
control Venezuela 5.03436745309
test Venezuela 4.8978308708
control Bolivia 4.93693693694
test Bolivia 4.79009687836
control Uruguay 1.20481927711
test Uruguay 1.29066953482
control Nicaragua 5.26469727991
test Nicaragua 5.41767554479
control Costa Rica 5.22556390977
test Costa Rica 5.47376368441
control Colombia 5.20894861193
test Colombia 5.05709624796
control Peru 4.99140435118
test Peru 5.06042745728
control Chile 4.81071754795
test Chile 5.12950222582
control Argentina 1.50705429671
test Argentina 1.37250180592
control Ecuador 4.91538078646
test Ecuador 4.89884209187
control El Salvador 5.35540408958
test El Salvador 4.79468895992
control Paraguay 4.84931506849
test Paraguay 4.92291046795
control Guatemala 5.06428758856
test Guatemala 4.86472077836
control Honduras 5.09057555607
test Honduras 4.75398145947
control Panama 4.67955239064
test Panama 4.93702770781


In [92]:
control_successes = control_df2.groupby('country').sum()

In [98]:
control_successes.loc['Argentina']['conversion']

141

In [ ]:
test_df2[test_df2['country']=='Spain']